# Example

## Using Prepared JSON-id Files

In [1]:
%%capture
import os
import sys
import pandas as pd
from dhis2api.country import Country

from dotenv import load_dotenv
load_dotenv()

from datetime import datetime

In [2]:
today = datetime.today().strftime("%m-%d-%Y")

In [3]:
json_fp = sys.path[0] + "/../JsonSelection_All_DataElements.xlsx"
json_id_df = pd.read_excel(json_fp)
display(json_id_df.head(5))

,country,id,displayName,group,count or percentage,category,category_old,indicatortype,project,initiation,pdo,notes,Core,Annual report,Used Previously
0,afghanistan,A0D0ThB17vR,NMC-GEN- Functionality of electricity source a...,dataElements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,afghanistan,a0iYmnDD2RH,Cleft palate and cleft lip/ref,dataElements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,afghanistan,A0NYImqKtiU,GBV- Willingness to take legal action against ...,dataElements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,afghanistan,a0v6xlpNqwE,TBIS- Percentage of HIV positive new and relap...,indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,afghanistan,a0ZLbjWKGZq,HIS- Percentage of CHCs with ambulance transpo...,indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
SLAuth = (os.getenv("SLUsername"), os.getenv("SLPassword"))

country_obj = Country(country="sierraleone",
                      base_url= 'https://sl.dhis2.org/hmis/api/29/analytics.json?', 
                      dates=["202201"], level="LEVEL-5",
                      auth=SLAuth,
                      idfilepath=json_fp)
country_obj.request_and_save()


ownloading sierraleone: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:17<00:00,  1.12s/it]

In [5]:
df = pd.read_csv(f"./sierraleone_{today}.csv")
sum_df = df.groupby("dataid")["Total"].sum().reset_index()
display(sum_df)

,dataid,Total
0,F5DCeEkMZbP,23905.0
1,F63VoBCwxxX,25406.0
2,JTZbSlVJnmv,187181.0
3,Q1MX1awbz1h,5695.0
4,Ra0KDOCDTBb,7527.0
5,TVOY9C29ikm,3208.0
6,UNNPnnylzkn,935.0
7,W2eW51uqgPP,16844.0
8,XtpblNP6IIf,16.0
9,grudkdawI9h,19841.0


## Direct Using pre-loaded JSON file

In [6]:
single_download = Country(country="sierraleone",
                          base_url= 'https://sl.dhis2.org/hmis/api/29/analytics.json?', 
                          dates=["202201"], level="LEVEL-5",
                          indicators = ["F5DCeEkMZbP"],
                          auth=SLAuth)
single_download.request_and_save()


ownloading sierraleone: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.02s/it]

In [12]:
single_df = pd.read_csv(f"./sierraleone_{today}.csv")
single_sum_df = single_df.groupby("dataid")["Total"].sum().reset_index()
display(single_sum_df)

,dataid,Total
0,F5DCeEkMZbP,23905.0


## Disaggregation

In [26]:
disaggregate_dims = ["BUbWS5DCgXK", "nB1SNlTWQJQ"]

disaggregate_download = Country(country="sierraleone",
                                base_url= 'https://sl.dhis2.org/hmis/api/29/analytics.json?', 
                                dates=["202201"], level="LEVEL-5",
                                indicators=["F5DCeEkMZbP"],
                                disaggregate=disaggregate_dims,
                                auth=SLAuth)
disaggregate_download.request_and_save()


ownloading sierraleone: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.05s/it]

In [27]:
dis_id_str = "".join(d for d in disaggregate_dims)
disaggregate_df = pd.read_csv(f"./sierraleone_{dis_id_str}_{today}.csv")

## Group by service point and age
double_disaggregate_df = (
    disaggregate_df.groupby(["dataid", "nB1SNlTWQJQname", "BUbWS5DCgXKname"])["Total"]
    .sum()
    .reset_index()
)

display(double_disaggregate_df)

,dataid,nB1SNlTWQJQname,BUbWS5DCgXKname,Total
0,F5DCeEkMZbP,In_Facility_X,0-11m_X,18149.0
1,F5DCeEkMZbP,In_Facility_X,12-59m_X,107.0
2,F5DCeEkMZbP,Outreach_X,0-11m_X,5534.0
3,F5DCeEkMZbP,Outreach_X,12-59m_X,115.0
